# AI05a: Introduction to APIs
## Walkthrough

**AI/ML Course | Medina County Career Center**

---

In this walkthrough, you'll learn to:
1. **Make HTTP requests** using the `requests` library
2. **Parse JSON responses** into Python data structures
3. **Use API libraries** that simplify common tasks

We'll use three different APIs:
- **Coinbase API** - Free cryptocurrency price data (raw HTTP)
- **USGS Earthquake API** - Real-time earthquake data from the US Geological Survey (raw HTTP)
- **NBA API** - Basketball statistics (using a library)

---

##  Setup

First, we need to install the libraries we'll use. Run these once, then you can comment them out.

In [ ]:
# Run this cell once to install required libraries
# After installation, you can comment these lines out

#! pip install requests     # For making HTTP requests
#! pip install nba-api      # For NBA statistics


Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [7]:
# Import the libraries we'll use throughout this notebook

import requests          # The main library for making HTTP requests
import json              # For pretty-printing JSON (built into Python)
import pandas as pd      # For working with data in tables
import time              # For adding delays between API calls
from datetime import datetime, timedelta  # For working with dates/times

print("All libraries imported successfully!")

All libraries imported successfully!


---

# Part 1: HTTP Requests with the `requests` Library

The `requests` library lets you make HTTP calls to any API. 

We'll start with the **Coinbase API** because:
- It's completely free
- No API key required
- Returns cryptocurrency price data

Documentation: https://docs.cloud.coinbase.com/exchange/reference/

## 1.1 Your First API Call - Bitcoin Price

Notice that this request does not use any parameters. In this API, the information we want is built directly into the URL itself. Some companies design their APIs this way, while others use parameters to customize requests. Both approaches are valid.

In [8]:
# ==============================================================================
# YOUR FIRST API CALL - Get Bitcoin Price
# ==============================================================================

# Step 1: Define the URL (endpoint) we want to call
# This is like the "address" of the data we want
url = "https://api.coinbase.com/v2/prices/BTC-USD/spot"

# Step 2: Make a GET request (asking for data)
# This sends a request to the Coinbase server and waits for a response
response = requests.get(url)

# Step 3: Check the status code
# 200 = Success! Anything else = something went wrong
print(f"Status Code: {response.status_code}")

# Step 4: Look at the raw response (it's JSON text)
print(f"Raw Response: {response.text}")

# Step 5: Convert JSON to Python dictionary
# This is where we employ response.json() function to turn the JSON into a dict we can work with!
data = response.json()
print(f"\nAs Python dict: {data}")
print(f"Type: {type(data)}")

Status Code: 200
Raw Response: {"data":{"amount":"66471.35","base":"BTC","currency":"USD"}}

As Python dict: {'data': {'amount': '66471.35', 'base': 'BTC', 'currency': 'USD'}}
Type: <class 'dict'>


### What did we just do?

1. We defined a URL pointing to Coinbase's Bitcoin price endpoint
2. `requests.get()` sent an HTTP GET request to that URL
3. The server responded with a status code (200 = success)
4. The response body contained JSON text
5. `response.json()` converted that JSON into a Python dictionary

**This is the core pattern for ALL API calls!**

## 1.2 Extracting Specific Data from the Response

Now let's pull out the actual price value and format it nicely.

In [9]:
# ==============================================================================
# EXTRACT AND FORMAT THE PRICE
# ==============================================================================

# Make the API call
url = "https://api.coinbase.com/v2/prices/BTC-USD/spot"   # try some different currencies here (e.g. EUR, GBP...)
response = requests.get(url)

# Check if the request worked (200 means "success")
if response.status_code == 200:
    
    # Convert the JSON text response into a Python dictionary
    data = response.json()
    
    # Access values from the nested API response dictionary
    price = float(data["data"]["amount"])   # Price comes in as text, so we convert it
    currency = data["data"]["currency"]     # Currency label (USD)
    
    # Print the price nicely formatted with commas and 2 decimals
    print(f"Bitcoin Price: ${price:,.2f} {currency}")

else:
    # This runs if the request failed
    print(f"Error! Status code: {response.status_code}")
    print(f"Response: {response.text}")

Bitcoin Price: $66,492.01 USD


## Let's just run the dictionary after it's returned to see the dictionary inside the dictionary: 

In [11]:
data

{'data': {'amount': '66492.015', 'base': 'BTC', 'currency': 'USD'}}

## 1.3 Getting Multiple Cryptocurrency Prices

Let's get prices for Bitcoin, Ethereum, and Solana by making multiple API calls.

In [12]:
# ================================
# Get Multiple Cryptocurrency Prices
# ================================

cryptos = ["BTC", "ETH", "SOL"]                            # Cryptocurrencies to look up
prices = []                                                # Empty list to store results

# ================================
# Loop Through Each Cryptocurrency
# ================================

for crypto in cryptos:                                     # Run once per crypto symbol

    url = f"https://api.coinbase.com/v2/prices/{crypto}-USD/spot"  # Build API URL (BTC-USD, ETH-USD, etc.)
    response = requests.get(url)                           # Send request to Coinbase API

    if response.status_code == 200:                        # 200 = request worked
        data = response.json()                             # Convert JSON to Python dictionary
        amount = float(data["data"]["amount"])             # Extract price and convert to number

        prices.append({                                   # Store one row of data
            "Crypto": crypto,                              # Crypto symbol
            "Price (USD)": amount                          # Current USD price
        })

    time.sleep(0.5)                                        # Small delay to avoid spamming the API

# ================================
# Display Results
# ================================

df = pd.DataFrame(prices)                                  # Convert list of dicts to DataFrame
print("\nCryptocurrency Prices:")
print(df.to_string(index=False))                           # Print table without row numbers



Cryptocurrency Prices:
Crypto  Price (USD)
   BTC    66989.705
   ETH     1966.700
   SOL       83.365


## What if I want to add more crypto in the above response? 

https://api.coinbase.com/v2/currencies  # run this in the browser and you'll get a JSON repsonse

---

# Part 2: A More Complex API - USGS Earthquakes

Now let's work with the **USGS Earthquake API**, which provides real-time earthquake data from around the world.

This API is more complex because:
- It has many optional parameters
- Returns nested, complex JSON structures
- Provides geospatial data (latitude/longitude)

Documentation: https://earthquake.usgs.gov/fdsnws/event/1/

## 2.1 Get Recent Significant Earthquakes

Let's find all earthquakes magnitude 5.0+ from the past week.

In [13]:
# ================================
# USGS Earthquake API Setup
# ================================

url = "https://earthquake.usgs.gov/fdsnws/event/1/query"   # Base API endpoint (where we send the request)

params = {                                                # Extra settings sent with the request
    "format": "geojson",      # Ask for JSON data in GeoJSON format
    "minmagnitude": 5.0,      # Only earthquakes magnitude 5.0+
    "orderby": "time",        # Newest earthquakes first
    "limit": 10               # Return only the 10 most recent
}

# ================================
# Send Request to the API
# ================================

response = requests.get(url, params=params)               # Send GET request with parameters

# ================================
# Handle the Response
# ================================

if response.status_code == 200:                            # 200 = request worked
    data = response.json()                                # Convert JSON response to Python dictionary

    print(f"Total earthquakes found: {data['metadata']['count']}")   # Summary count
    print("\nRecent significant earthquakes (magnitude 5.0+):\n")
    print("=" * 80)

    for i, quake in enumerate(data['features'], 1):       # Loop through each earthquake
        props = quake['properties']                       # Earthquake details (magnitude, place, time)
        coords = quake['geometry']['coordinates']         # Longitude, latitude, depth

        magnitude = props['mag']                           # Earthquake magnitude
        place = props['place']                             # Location description
        time_ms = props['time']                            # Time in milliseconds since 1970

        quake_time = datetime.fromtimestamp(time_ms / 1000)  # Convert to readable date

        print(f"{i}. Magnitude {magnitude} - {place}")     # Main summary line
        print(f"   Time: {quake_time.strftime('%Y-%m-%d %H:%M:%S')}")  # Formatted date
        print(f"   Coordinates: {coords[1]:.2f}°N, {coords[0]:.2f}°E")  # Latitude, longitude
        print()

else:
    print(f"Error! Status code: {response.status_code}")   # API request failed


Total earthquakes found: 10

Recent significant earthquakes (magnitude 5.0+):

1. Magnitude 5.5 - 105 km NE of Tatsugō, Japan
   Time: 2026-02-04 13:20:42
   Coordinates: 29.11°N, 130.39°E

2. Magnitude 6.1 - Kermadec Islands, New Zealand
   Time: 2026-02-04 05:39:29
   Coordinates: -29.60°N, -178.60°E

3. Magnitude 5.1 - 8 km SW of Kimbe, Papua New Guinea
   Time: 2026-02-03 23:25:54
   Coordinates: -5.60°N, 150.08°E

4. Magnitude 5.3 - south of the Fiji Islands
   Time: 2026-02-03 21:07:44
   Coordinates: -26.94°N, -176.28°E

5. Magnitude 5.1 - 153 km SSW of Tual, Indonesia
   Time: 2026-02-03 19:36:31
   Coordinates: -6.83°N, 132.05°E

6. Magnitude 5.2 - 101 km NE of Hihifo, Tonga
   Time: 2026-02-03 14:34:01
   Coordinates: -15.34°N, -173.09°E

7. Magnitude 5.2 - 49 km NW of Finschhafen, Papua New Guinea
   Time: 2026-02-03 14:21:36
   Coordinates: -6.21°N, 147.56°E

8. Magnitude 5.1 - 89 km W of Yenangyaung, Burma (Myanmar)
   Time: 2026-02-03 10:51:31
   Coordinates: 20.56°N, 94.

## 2.2 Create a DataFrame of Earthquake Data

Let's convert the earthquake data into a pandas DataFrame for easier analysis.

In [14]:
# ================================
# Convert Earthquake Data to DataFrame
# ================================

url = "https://earthquake.usgs.gov/fdsnws/event/1/query"     # USGS earthquake API endpoint
params = {                                                   # Request settings
    "format": "geojson",                                     # Return data as JSON
    "minmagnitude": 5.0,                                     # Only magnitude 5.0+
    "orderby": "time",                                       # Newest first
    "limit": 20                                              # Get more rows for DataFrame
}

response = requests.get(url, params=params)                 # Send API request

# ================================
# Process API Response
# ================================

if response.status_code == 200:                              # Check if request worked
    data = response.json()                                   # Convert JSON to Python dict

    earthquakes = []                                         # Empty list to store rows

    for quake in data['features']:                           # Loop through each earthquake
        props = quake['properties']                          # Earthquake details
        coords = quake['geometry']['coordinates']            # Longitude, latitude, depth

        earthquakes.append({                                 # Build one row of data
            'Magnitude': props['mag'],                       # Earthquake magnitude
            'Location': props['place'],                     # Location description
            'Time': datetime.fromtimestamp(props['time'] / 1000),  # Convert timestamp
            'Latitude': coords[1],                           # Latitude
            'Longitude': coords[0],                          # Longitude
            'Depth (km)': coords[2]                          # Depth in kilometers
        })

# ================================
# Create & Display DataFrame
# ================================

    quake_df = pd.DataFrame(earthquakes)                     # Convert list to DataFrame

    print("Recent Earthquakes (Magnitude 5.0+)\n")
    display(quake_df)                                        # Display table (Jupyter)

# ================================
# Basic Statistics
# ================================

    print("\nStatistics:")
    print(f"Average magnitude: {quake_df['Magnitude'].mean():.2f}")  # Mean magnitude
    print(f"Maximum magnitude: {quake_df['Magnitude'].max():.2f}")   # Largest quake
    print(f"Average depth: {quake_df['Depth (km)'].mean():.2f} km")  # Mean depth


Recent Earthquakes (Magnitude 5.0+)



,Magnitude,Location,Time,Latitude,Longitude,Depth (km)
0,5.5,"105 km NE of Tatsugō, Japan",2026-02-04 13:20:42.574,29.1074,130.3853,21.435
1,6.1,"Kermadec Islands, New Zealand",2026-02-04 05:39:29.988,-29.5951,-178.5952,184.162
2,5.1,"8 km SW of Kimbe, Papua New Guinea",2026-02-03 23:25:54.839,-5.6004,150.0844,143.776
3,5.3,south of the Fiji Islands,2026-02-03 21:07:44.796,-26.9354,-176.2788,10.000
4,5.1,"153 km SSW of Tual, Indonesia",2026-02-03 19:36:31.113,-6.8285,132.0522,11.096
5,5.2,"101 km NE of Hihifo, Tonga",2026-02-03 14:34:01.908,-15.3423,-173.0865,10.000
6,5.2,"49 km NW of Finschhafen, Papua New Guinea",2026-02-03 14:21:36.973,-6.2150,147.5575,72.189
7,5.1,"89 km W of Yenangyaung, Burma (Myanmar)",2026-02-03 10:51:31.021,20.5627,94.0152,71.111
8,5.1,western Indian-Antarctic Ridge,2026-02-03 10:38:50.177,-51.3489,139.1004,10.000
9,5.8,"97 km W of Yenangyaung, Burma (Myanmar)",2026-02-03 10:34:01.298,20.4466,93.9337,56.995



Statistics:
Average magnitude: 5.28
Maximum magnitude: 6.10
Average depth: 49.30 km


### Understanding the URL Structure

When we pass `params` to `requests.get()`, it builds the full URL for us:

```
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&minmagnitude=5.0&orderby=time&limit=20
```

Let's see what URL was actually called:

In [15]:
# Show the actual URL that was constructed
print("Full URL that was called:")
print(response.url)

Full URL that was called:
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&minmagnitude=5.0&orderby=time&limit=20


---

# Summary

## What You Learned:

1. **APIs** are interfaces for programs to exchange data
2. **HTTP GET requests** retrieve data from servers
3. **JSON** is the standard data format (like Python dicts)
4. **`requests` library** makes raw HTTP calls
5. **Status codes** tell you if requests succeeded (200 = OK)
6. **Parameters** customize API requests
7. **Error handling** is essential for robust code

## Key Code Patterns:

```python
# Raw HTTP request
response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
```

## APIs We Explored:

1. **Coinbase** - Cryptocurrency prices (raw HTTP)
2. **USGS Earthquakes** - Real-time earthquake data (raw HTTP)